In [ ]:
from twitter import scraper
from twitter.login import login
from twitter.utils import find_key

In [ ]:
username,password = ...,...
s = login(username, password)

In [ ]:
tweets = scraper.get_user_tweets(s, [123,234,345,456,567])

#### Find all unique urls in users tweets

In [ ]:
unique_urls = set(find_key(tweets,'expanded_url'))
unique_urls